# Demand Forecasting (AI, Data Innovation Team, CJ Express)

## File: Training Model (Cluster Model)

 C.J. Express Group Co.,Ltd. All Rights Reserved.
 
 Author: Kao Panboonyuen, Ph.D.
 
 Year: 2021
 ![](https://github.com/kaocj/demandforecasting_darkside/raw/main/demand_main.PNG)

In [10]:
!rm -rf cjdarkside.py
!wget https://github.com/kaocj/demandforecasting_darkside/raw/main/cjdarkside.py -q --show-progress

cjdarkside.py       100%[===================>]  45.18K  --.-KB/s    in 0.002s  


In [11]:
spark

# Import Library

In [12]:
from cjdarkside import *

# Name your CJ DARKSIDE model

In [14]:
version = '1_DT_Based'
versionPL = 'PL_Feature_v2'

gs_model = "gs://cj-demand-forecasting-dataset/model/2021-April-v1/"
gs_csv = "gs://cj-demand-forecasting-dataset/csv/"

PL_FEATURES_PATH = gs_model+"demand_cjdarkside_pl_features_2021_v"+str(versionPL)+".plmodel"

PL_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_pl_c_binary_model_2021_v"+str(version)+".plmodel"
ML_MODEL_Classification_PATH = gs_model+"demand_cjdarkside_ml_c_binary_model_2021_v"+str(version)+".model"

PL_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_pl_r_binary_model_2021_v"+str(version)+".plmodel"
ML_MODEL_Regression_PATH = gs_model+"demand_cjdarkside_ml_r_binary_model_2021_v"+str(version)+".model"

# Data Set (Input)

In [15]:
DATASET_PATH = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v3/part-0*"
DATASET_PATH_JUST_ONE = "gs://cj-demand-forecasting-dataset/demand_prediction_pipeline/all_sku_2020_v2/part-00001-35019035-6748-4de5-885e-8e30b63ef12e-c000.csv"

In [16]:
branch_location = 'gs://cj-demand-forecasting-dataset/_additional/BranchDistrictTH_EN_2.csv'
welfare_location = 'gs://cj-demand-forecasting-dataset/_additional/csv_branch_welfare.csv'
reference_qty = 'gs://cj-demand-forecasting-dataset/_reference2/reference00*'
flag_qty = 'gs://cj-demand-forecasting-dataset/_additional/flag_sku_branch.csv'
price_sens = "gs://cj-demand-forecasting-dataset/ad_data/ps_agg_v2_0*"
tier_cj = "gs://cj-demand-forecasting-dataset/tier/tier00000*"

In [17]:
select_variables_initial = ['cj_darkside_id','SalDate','BranchCode','MaterialCode','avgPriceDis',\
                            'avgPrice','supPrice','label','totalNetSale',\
                            'TotalQtySale','types','ZipCode','ProvinceEN','DistrictEN',\
                            'DayofMonth','Yearday','Month','DayofWeek','Year',\
                            'Quarter','WeekOfYear','MonthQuarter','Branch','Name']

# CREATE TRAIN SET

In [ ]:
df = spark.read.format("csv").option("header", "true").load(DATASET_PATH)
df = clean_basic(df)

print('-----------------------------------')
print('-- welcome to the darkside of cj --')
print('-----------------------------------')

df.cache()

-----------------------------------
-- welcome to the darkside of cj --
-----------------------------------
count: before sampling:  1134005882
count: after sampling:  1134005882


'\nsampling = 0.3\ncount: before sampling:  1372,654,452\ncount: after sampling:  411,761,580\n\nsampling = 0.03\ncount: before sampling:  1,372,654,452\ncount: after sampling:  41,761,580\n\nsampling = 0.003\ncount: before sampling:  1,372,654,452\ncount: after sampling:  4,761,580\n\n'

# Start making Features

In [ ]:
# Step1) Create Feature: Location

branch_location_df = spark.read.format("csv").option("header", "true").load(branch_location)
branch_location_df = branch_location_df.withColumnRenamed("BranchCode","BranchCodeLocation")

df = df.join(branch_location_df, df.BranchCode == branch_location_df.BranchCodeLocation)

# Step2) Create Feature: All Play with Date and Welfare

df = df.select("*").withColumn("cj_darkside_id", monotonically_increasing_id())

df = df.withColumnRenamed("Date","SalDate")
df = df.withColumn('DayofMonth', F.dayofmonth(F.col('SalDate')))

df = extract_date(df)

df = select_variable(df, select_variables_initial)

wf = read_file(welfare_location,'csv')
df = transform_welfare_flag(df, wf)
df = transform_welfare_flag_day_2(df)

df = df.withColumnRenamed("types","promotion")

df = df.dropna(how='all')

# Feature: ONE HOT ENCODER

In [ ]:
# load features pipeline
pipeline_features = Pipeline.read().load(PL_FEATURES_PATH)
df_transformed = pipeline_features.fit(df).transform(df)

# Main (Feature Engineering)

In [ ]:
# Show your previous darkside feature -> Here !!!

In [ ]:
final_df = clean_binary_2(df_transformed)
final_df.cache()

DataFrame[cj_darkside_id: bigint, SalDate: string, BranchCode: int, MaterialCode: int, avgPriceDis: double, avgPrice: double, supPrice: double, label: int, totalNetSale: double, TotalQtySale: int, promotion: string, ZipCode: int, ProvinceEN: string, DistrictEN: string, DayofMonth: int, Yearday: int, Month: int, DayofWeek: int, Year: int, Quarter: int, WeekOfYear: int, MonthQuarter: int, Branch: string, Name: string, welfareFlag: int, welfareFlagDay: int, BranchCode_idx: double, BranchCode_vec: vector, MaterialCode_idx: double, MaterialCode_vec: vector, promotion_idx: double, promotion_vec: vector, ProvinceEN_idx: double, ProvinceEN_vec: vector, DistrictEN_idx: double, DistrictEN_vec: vector]

# Feature: High-Q

In [ ]:
###########################################################################
############### Create Reference Quality Variable #########################
###########################################################################

reference_qty_df = spark.read.format("csv").option("header", "true").load(reference_qty)
reference_qty_df = reference_qty_df.withColumnRenamed("BranchCode","BranchRefQty").withColumnRenamed("MaterialCode","SKURefQty")

reference_qty_df = reference_qty_df.withColumn("BranchRefQty", reference_qty_df["BranchRefQty"].cast("integer"))
reference_qty_df = reference_qty_df.withColumn("SKURefQty", reference_qty_df["SKURefQty"].cast("integer"))

final_df = final_df.join(reference_qty_df,(final_df.BranchCode == reference_qty_df.BranchRefQty) &\
                         (final_df.MaterialCode == reference_qty_df.SKURefQty) &\
                          (final_df.DayofWeek == reference_qty_df.DoW_Num))

###########################################################################
############### Create High Quality Flag Variable #########################
###########################################################################

flag_qty_df = spark.read.format("csv").option("header", "true").load(flag_qty)
#flag_qty_df = flag_qty_df.filter(flag_qty_df.BranchCode.like('%M%') == False)

flag_qty_df = flag_qty_df.dropna()
flag_qty_df = flag_qty_df.withColumn("BranchCode", flag_qty_df["BranchCode"].cast("integer"))
flag_qty_df = flag_qty_df.withColumn("MaterialCode", flag_qty_df["MaterialCode"].cast("integer"))

l_bc = [cj[0] for cj in flag_qty_df.select('BranchCode').distinct().toLocalIterator()]
l_mc = [cj[0] for cj in flag_qty_df.select('MaterialCode').distinct().toLocalIterator()]

matches = (  (F.col('BranchCode').isin(l_bc)) & (F.col('MaterialCode').isin(l_mc))   )
final_df = final_df.withColumn("HighQualityFlag", when(matches, "1").otherwise("0"))

final_df = clean_binary_4(final_df)

# GBT ML Model (Classifier)

In [ ]:
############ ----- LOAD ML MODEL ----- ############

read_plmodel = PipelineModel.read().load(PL_MODEL_Classification_PATH)
print (read_plmodel.stages)

read_model = GBTRegressor.read().load(ML_MODEL_Classification_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsC = read_plmodel.transform(final_df)
predictionsC.cache()


print("Testing Process: Done")

VectorAssembler_1c3584edf7bc
DecisionTreeClassifier_9b560e21d3cc
[VectorAssembler_1c3584edf7bc, DecisionTreeClassificationModel (uid=DecisionTreeClassifier_9b560e21d3cc) of depth 5 with 33 nodes]
Training Process: Done


# GBT ML Model (Regressor)

In [ ]:
############ ----- LOAD ML MODEL ----- ############

read_plmodel = PipelineModel.read().load(PL_MODEL_Regression_PATH)
print (read_plmodel.stages)

read_model = GBTRegressor.read().load(ML_MODEL_Regression_PATH)

############ ----- DONE ML MODEL ----- ############

predictionsR = read_plmodel.transform(final_df)
predictionsR.cache()


print("Testing Process: Done")

VectorAssembler_03b7ccc7005f
DecisionTreeRegressor_ba0c7a9f7ba7
[VectorAssembler_03b7ccc7005f, DecisionTreeRegressionModel (uid=DecisionTreeRegressor_ba0c7a9f7ba7) of depth 5 with 63 nodes]
Training Process: Done


# FINAL PREDICTION (C + R)

In [ ]:
predictionsC.show(2)

+--------------+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|BranchCode_idx|

In [ ]:
predictionsR.show(2)

+--------------+----------+----------+------------+-----------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+------------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|Quarter|WeekOfYear|MonthQuarter|             Branch|                Name|welfareFlag|welfareFlagDay|BranchCode_idx|   BranchCode_vec|MaterialCode_idx|   

In [ ]:
def ansByProb(p, t=0.4):
  if p>=t: return 1
  elif p<t: return 0
  else: 0

predictionsA = predictionsC.join(predictionsR,(predictionsC.SalDate == predictionsR.SalDate)&\
                  (predictionsC.BranchCode == predictionsR.BranchCode)&\
                  (predictionsC.MaterialCode == predictionsR.MaterialCode)).select(predictionsC["*"],predictionsR["predict_qty_lv2"])

split1_udf = udf(lambda value: value[0].item(), FloatType())
split2_udf = udf(lambda value: value[1].item(), FloatType())

predictionsA = predictionsA.withColumn('probability_class0', split1_udf('probability')).withColumn('probability_class1', split2_udf('probability'))

ansByP = F.udf(ansByProb, IntegerType())
predictionsA = predictionsA.withColumn("predict_bi_lv1_with_thresh", ansByP("probability_class1"))

predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1")*col("predict_qty_lv2"))
#predictionsA = predictionsA.withColumn("predict_qty_final", col("predict_bi_lv1_with_thresh")*col("predict_qty_lv2"))

In [ ]:
predictionsA.show(2)

+--------------+----------+----------+------------+------------------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------+-----------------+------------------+------------------+--------------------------+-----------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|       avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearday|Month|DayofWeek|Year|

# POST PROCESSING (WITH TIME SERIES)

In [ ]:
#predictionsA = post_score_v3(predictionsA)
predictionsA.show(2)

+--------------+----------+----------+------------+------------------+--------+--------+-----+------------+------------+---------+-------+-----------+----------+----------+-------+-----+---------+----+-------+----------+------------+-------------------+--------------------+-----------+--------------+--------------+-----------------+----------------+-------------------+-------------+---------------+--------------+--------------+--------------+----------------+------------+---------+---+-------+----------------+--------------+--------------+-------------+-----------+-------------+-----------------+---------------+--------------------+--------------------+--------------------+--------------+-----------------+------------------+------------------+--------------------------+-----------------+----------------------+
|cj_darkside_id|   SalDate|BranchCode|MaterialCode|       avgPriceDis|avgPrice|supPrice|label|totalNetSale|TotalQtySale|promotion|ZipCode| ProvinceEN|DistrictEN|DayofMonth|Yearda

In [1]:
'Done'

'Done'

# Criteria 
![](https://github.com/kaocj/demandforecasting_darkside/raw/main/criteria.PNG)